In [0]:
from pyspark.sql.functions import*
from pyspark.sql.types import *

## Data Reading

In [0]:
df = spark.read.format("parquet").load("abfss://bronze@adlscardatalake.dfs.core.windows.net/products")

In [0]:
df.display()

In [0]:
df = df.drop('_rescued_data')
df.display()

In [0]:
df.createOrReplaceTempView("products")

## Functions

In [0]:
%sql
create or replace function retail_cat.bronze.discount_func(p_price double)
returns double
language sql
return p_price * 0.90

In [0]:
%sql
select product_id,retail_cat.bronze.discount_func(price) as discounted_price from products

In [0]:
df = df.withColumn("discounted_price",expr("retail_cat.bronze.discount_func(price)"))  #use expr to call function in any column like sql way
df.display()

In [0]:
%sql
create or replace function retail_cat.bronze.upper_func(p_brand string)
returns string
language python
as
$$

  return p_brand.upper()
  
$$

In [0]:
%sql
    
select product_id,brand, retail_cat.bronze.upper_func(brand) as brand_upper from products

In [0]:
df.write.mode("overwrite").format("delta").save("abfss://silver@adlscardatalake.dfs.core.windows.net/products")

In [0]:
%sql
create table if not exists retail_cat.silver_sch.products_silver
using delta
location "abfss://silver@adlscardatalake.dfs.core.windows.net/products"